# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [30]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json

# Import API key
from api_keys import geoapify_key

In [31]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,vestmannaeyjar,63.4427,-20.2734,275.70,75,40,11.32,IS,1674146345
1,1,rikitea,-23.1203,-134.9692,298.16,71,100,0.75,PF,1674145781
2,2,marienburg,54.0359,19.0266,275.95,78,100,3.02,PL,1674146110
3,3,thompson,55.7435,-97.8558,252.24,77,0,0.00,CA,1674145605
4,4,mataura,-46.1927,168.8643,284.51,99,86,1.06,NZ,1674145786


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [32]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    size = "Humidity"
)

# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [33]:
# Narrow down cities that fit criteria and drop any results with null values
my_city_data_df = city_data_df.loc[
    (city_data_df["Max Temp"] >275) & 
    (city_data_df["Max Temp"] <=290) & 
    (city_data_df["Wind Speed"] < 5) & 
    (city_data_df["Cloudiness"] < 50)]

# Drop any rows with null values
my_city_data_df.dropna()

# Display sample data
my_city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
23,23,kaiyuan,23.6977,103.3037,280.65,91,1,1.61,CN,1674146356
40,40,ribeira grande,38.5167,-28.7000,289.37,67,40,2.06,PT,1674146366
42,42,half moon bay,37.4636,-122.4286,281.90,89,40,0.00,US,1674146367
43,43,ushuaia,-54.8000,-68.3000,285.94,66,0,2.06,AR,1674146367
52,52,myitkyina,25.3833,97.4000,285.73,51,0,1.30,MM,1674146372


### Step 3: Create a new DataFrame called `hotel_df`.

In [34]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = my_city_data_df[["City", "Country", "Lat","Lng","Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ''

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
23,kaiyuan,CN,23.6977,103.3037,91,
40,ribeira grande,PT,38.5167,-28.7000,67,
42,half moon bay,US,37.4636,-122.4286,89,
43,ushuaia,AR,-54.8000,-68.3000,66,
52,myitkyina,MM,25.3833,97.4000,51,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [44]:
# just an example API call for reference

latitude = 40.415392
longitude = -3.7073743182788528

radius = 10000
limit = 1
category = "accommodation.hotel"

params = {
"categories":category,
"limit":limit,
"apiKey":geoapify_key, 
"filter": f"circle:{longitude},{latitude},{radius}",
"bias": f"proximity:{longitude},{latitude}"
}
    
# Set base URL
base_url = "https://api.geoapify.com/v2/places"


# Make and API request using the params dictionaty
name_address = requests.get(base_url,params=params)
    
# Convert the API response to JSON format
name_address = name_address.json()
    
print(json.dumps(name_address, indent=4, sort_keys=True))

{
    "features": [
        {
            "geometry": {
                "coordinates": [
                    -3.707301099999999,
                    40.415012800321946
                ],
                "type": "Point"
            },
            "properties": {
                "address_line1": "Hotel Pestana Plaza Mayor",
                "address_line2": "Calle Imperial, 8, 28012 Madrid, Spain",
                "categories": [
                    "accommodation",
                    "accommodation.hotel"
                ],
                "city": "Madrid",
                "country": "Spain",
                "country_code": "es",
                "county": "\u00c1rea metropolitana de Madrid y Corredor del Henares",
                "datasource": {
                    "attribution": "\u00a9 OpenStreetMap contributors",
                    "license": "Open Database Licence",
                    "raw": {
                        "contact:email": "guest@pestana.com",
                        "c

In [45]:
# Set parameters to search for a hotel

radius = 10000
limit = 1
category = "accommodation.hotel"

params = {
    "categories":category,
    "limit":limit,
    "apiKey":geoapify_key    
}


# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = hotel_df.loc[index, "Lat"]
    longitude = hotel_df.loc[index, "Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url,params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
kaiyuan - nearest hotel: No hotel found
ribeira grande - nearest hotel: Hospedaria JSF - 3 Estrelas
half moon bay - nearest hotel: Half Moon Bay Inn
ushuaia - nearest hotel: Apart Hotel Aires del Beagle
myitkyina - nearest hotel: New Light Hotel
new norfolk - nearest hotel: Woodbridge on the Derwent
mehriz - nearest hotel: هتل رستوران شاه عباسی
baghdad - nearest hotel: AL Bedaa Hotel
san marcos - nearest hotel: Fairfield Inn & Suites by Marriott San Diego North/San Marcos
santa ana - nearest hotel: Holiday Inn Express
rangapara - nearest hotel: No hotel found
lianran - nearest hotel: 金色时代酒店
nibbar - nearest hotel: No hotel found
ucar - nearest hotel: Otel
muroto - nearest hotel: 民宿うらしま
plouzane - nearest hotel: Les Voyageurs
pueblo - nearest hotel: Station on the Riverwalk
coos bay - nearest hotel: No hotel found
jiangyou - nearest hotel: No hotel found
tabas - nearest hotel: هتل امیر
kaeo - nearest hotel: No hotel found
palivere - nearest hotel: No hotel found
ha

,City,Country,Lat,Lng,Humidity,Hotel Name
23,kaiyuan,CN,23.6977,103.3037,91,No hotel found
40,ribeira grande,PT,38.5167,-28.7000,67,Hospedaria JSF - 3 Estrelas
42,half moon bay,US,37.4636,-122.4286,89,Half Moon Bay Inn
43,ushuaia,AR,-54.8000,-68.3000,66,Apart Hotel Aires del Beagle
52,myitkyina,MM,25.3833,97.4000,51,New Light Hotel
55,new norfolk,AU,-42.7826,147.0587,72,Woodbridge on the Derwent
66,mehriz,IR,31.5917,54.4318,78,هتل رستوران شاه عباسی
93,baghdad,IQ,33.3406,44.4009,77,AL Bedaa Hotel
105,san marcos,US,33.1434,-117.1661,69,Fairfield Inn & Suites by Marriott San Diego N...
112,santa ana,US,33.7456,-117.8678,84,Holiday Inn Express


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [60]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    c = "Country",
    hover_cols = 'all',
)


# Display the map
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Country,index,City,Humidity,Hotel Name)